# Prediction of used car prices 

Loading required libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

from sklearn.metrics import r2_score


# Import Data

In [2]:
df_tr=pd.read_csv('data_train.csv')
df_te=pd.read_csv('data_test.csv')
df_tr.shape, df_te.shape

((50000, 17), (27000, 17))

In [3]:
df_tr.head()

,manufacturer_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_type,engine_capacity,body_type,has_warranty,ownership,type_of_drive,is_exchangeable,number_of_photos,number_of_maintenance,duration_listed,price_usd
0,Volkswagen,automatic,black,130000,2016,diesel,diesel,1.6,universal,False,owned,front,True,17,38,67,13150.0
1,Renault,manual,brown,149000,2012,gasoline,gasoline,1.6,sedan,False,owned,front,False,9,3,100,7500.0
2,Kia,automatic,brown,110000,2014,gasoline,gasoline,1.6,hatchback,False,owned,front,False,5,10,91,12200.0
3,Opel,automatic,other,255100,2007,gasoline,gasoline,1.8,hatchback,False,owned,front,False,10,4,91,4950.0
4,Mazda,manual,blue,650000,1999,gasoline,gasoline,2.0,sedan,False,owned,front,True,5,7,62,3000.0


Checking null values

In [4]:
df_tr.isnull().sum()

manufacturer_name         0
transmission              0
color                     0
odometer_value            0
year_produced             0
engine_fuel               0
engine_type               0
engine_capacity          15
body_type                 0
has_warranty              0
ownership                 0
type_of_drive             0
is_exchangeable           0
number_of_photos          0
number_of_maintenance     0
duration_listed           0
price_usd                 0
dtype: int64

In [5]:
df_te.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          5
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

Dropping na values

In [6]:
dataTrain=df_tr.dropna()
dataTest=df_te.dropna()

In [7]:
dataTrain.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [8]:
dataTrain.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [9]:
dataTest.isnull().sum()

manufacturer_name        0
transmission             0
color                    0
odometer_value           0
year_produced            0
engine_fuel              0
engine_type              0
engine_capacity          0
body_type                0
has_warranty             0
ownership                0
type_of_drive            0
is_exchangeable          0
number_of_photos         0
number_of_maintenance    0
duration_listed          0
price_usd                0
dtype: int64

In [10]:
dataTrain.drop(['number_of_maintenance', 'duration_listed', 'engine_type','is_exchangeable'], axis = 1, inplace = True)

C:\Users\vinod\AppData\Local\Temp\ipykernel_19348\2786449281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTrain.drop(['number_of_maintenance', 'duration_listed', 'engine_type','is_exchangeable'], axis = 1, inplace = True)


In [11]:
dataTest.drop(['number_of_maintenance', 'duration_listed', 'engine_type','is_exchangeable'], axis = 1, inplace = True)

C:\Users\vinod\AppData\Local\Temp\ipykernel_19348\147954485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataTest.drop(['number_of_maintenance', 'duration_listed', 'engine_type','is_exchangeable'], axis = 1, inplace = True)


In [12]:
dataTrain.shape, dataTest.shape

((49985, 13), (26995, 13))

In [13]:
dataTrain.dtypes

manufacturer_name     object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_capacity      float64
body_type             object
has_warranty            bool
ownership             object
type_of_drive         object
number_of_photos       int64
price_usd            float64
dtype: object

In [14]:
dataTrain.describe()

,odometer_value,year_produced,engine_capacity,number_of_photos,price_usd
count,49985.000000,49985.000000,49985.000000,49985.000000,49985.000000
mean,249275.754426,2002.942143,2.056894,9.660958,6651.131653
std,136342.510114,8.065809,0.675106,6.144711,6462.573155
min,0.000000,1942.000000,0.200000,1.000000,1.000000
25%,158000.000000,1998.000000,1.600000,5.000000,2100.000000
50%,250000.000000,2003.000000,2.000000,8.000000,4800.000000
75%,325000.000000,2009.000000,2.300000,12.000000,8963.290000
max,1000000.000000,2019.000000,8.000000,86.000000,50000.000000


In [15]:
dataTest.describe()

,odometer_value,year_produced,engine_capacity,number_of_photos,price_usd
count,26995.000000,26995.000000,26995.000000,26995.000000,26995.000000
mean,248253.283497,2002.935655,2.052027,9.623597,6610.308694
std,135573.549451,8.063814,0.663980,5.997012,6357.865198
min,0.000000,1942.000000,0.200000,1.000000,1.000000
25%,157000.000000,1998.000000,1.600000,5.000000,2150.000000
50%,249000.000000,2003.000000,2.000000,8.000000,4799.000000
75%,324000.000000,2009.000000,2.300000,12.000000,8950.000000
max,1000000.000000,2019.000000,8.000000,71.000000,50000.000000


Data preprosessing 

In [16]:
X_train = pd.get_dummies(dataTrain, drop_first=True)
X_test = pd.get_dummies(dataTest, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Converting boolen to int

In [18]:
# Identify boolean columns
bool_columns = X_train.select_dtypes(include='bool').columns

# Convert those columns to integers
X_train[bool_columns] = X_train[bool_columns].astype(int)
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49985 entries, 0 to 49999
Data columns (total 91 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   49985 non-null  int64  
 1   year_produced                    49985 non-null  int64  
 2   engine_capacity                  49985 non-null  float64
 3   has_warranty                     49985 non-null  int32  
 4   number_of_photos                 49985 non-null  int64  
 5   price_usd                        49985 non-null  float64
 6   manufacturer_name_Alfa Romeo     49985 non-null  int32  
 7   manufacturer_name_Audi           49985 non-null  int32  
 8   manufacturer_name_BMW            49985 non-null  int32  
 9   manufacturer_name_Buick          49985 non-null  int32  
 10  manufacturer_name_Cadillac       49985 non-null  int32  
 11  manufacturer_name_Chery          49985 non-null  int32  
 12  manufacturer_name_Chevr

In [19]:
# Identify boolean columns
bool_columns = X_test.select_dtypes(include='bool').columns

# Convert those columns to integers
X_test[bool_columns] = X_test[bool_columns].astype(int)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26995 entries, 0 to 26999
Data columns (total 91 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   26995 non-null  int64  
 1   year_produced                    26995 non-null  int64  
 2   engine_capacity                  26995 non-null  float64
 3   has_warranty                     26995 non-null  int32  
 4   number_of_photos                 26995 non-null  int64  
 5   price_usd                        26995 non-null  float64
 6   manufacturer_name_Alfa Romeo     26995 non-null  int32  
 7   manufacturer_name_Audi           26995 non-null  int32  
 8   manufacturer_name_BMW            26995 non-null  int32  
 9   manufacturer_name_Buick          26995 non-null  int32  
 10  manufacturer_name_Cadillac       26995 non-null  int32  
 11  manufacturer_name_Chery          26995 non-null  int32  
 12  manufacturer_name_Chevr

In [20]:
X_train1=X_train.drop(columns=['price_usd'])
y_train=X_train['price_usd']
#test set
X_test1=X_test.drop(columns=['price_usd'])
y_test=X_test['price_usd']

In [21]:
X_train1.shape, y_train.shape, X_test1.shape, y_test.shape

((49985, 90), (49985,), (26995, 90), (26995,))

In [22]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train1, y_train, test_size=0.05, random_state=42)


In [23]:
X_train_split.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47485 entries, 25051 to 15799
Data columns (total 90 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   odometer_value                   47485 non-null  int64  
 1   year_produced                    47485 non-null  int64  
 2   engine_capacity                  47485 non-null  float64
 3   has_warranty                     47485 non-null  int32  
 4   number_of_photos                 47485 non-null  int64  
 5   manufacturer_name_Alfa Romeo     47485 non-null  int32  
 6   manufacturer_name_Audi           47485 non-null  int32  
 7   manufacturer_name_BMW            47485 non-null  int32  
 8   manufacturer_name_Buick          47485 non-null  int32  
 9   manufacturer_name_Cadillac       47485 non-null  int32  
 10  manufacturer_name_Chery          47485 non-null  int32  
 11  manufacturer_name_Chevrolet      47485 non-null  int32  
 12  manufacturer_name_C

In [24]:
X_train_split.shape,y_train_split.shape,X_val_split.shape,y_val_split.shape,X_test1.shape,y_test.shape

((47485, 90), (47485,), (2500, 90), (2500,), (26995, 90), (26995,))

Model building

In [25]:
tf.random.set_seed(42)

# Build the model
model = tf.keras.Sequential()

# Input layer (automatically determined by input shape)
model.add(tf.keras.layers.InputLayer(input_shape=(X_train1.shape[1],)))

# Hidden layers\
model.add(tf.keras.layers.Dense(400, activation='relu'))

model.add(tf.keras.layers.Dense(400, activation='relu'))
model.add(tf.keras.layers.Dense(200, activation='relu'))  
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(150, activation='relu'))
model.add(tf.keras.layers.Dense(150, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))


# Output layer
model.add(tf.keras.layers.Dense(1,activation='relu'))  # For regression, no activation function is used for the output layer

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])


In [26]:
# Train the model
history = model.fit(X_train_split, y_train_split, 
                    epochs=80, 
                    batch_size=256, 
                    validation_data=(X_val_split, y_val_split))


Epoch 1/80
186/186 [==============================] - 4s 7ms/step - loss: 5316.9590 - mae: 5316.9590 - val_loss: 5405.2246 - val_mae: 5405.2246
Epoch 2/80
186/186 [==============================] - 1s 7ms/step - loss: 4159.0132 - mae: 4159.0132 - val_loss: 4712.0640 - val_mae: 4712.0640
Epoch 3/80
186/186 [==============================] - 1s 7ms/step - loss: 4001.1028 - mae: 4001.1028 - val_loss: 4518.7598 - val_mae: 4518.7598
Epoch 4/80
186/186 [==============================] - 1s 6ms/step - loss: 3907.0710 - mae: 3907.0710 - val_loss: 4245.0498 - val_mae: 4245.0498
Epoch 5/80
186/186 [==============================] - 1s 6ms/step - loss: 3898.5142 - mae: 3898.5142 - val_loss: 4172.6147 - val_mae: 4172.6147
Epoch 6/80
186/186 [==============================] - 1s 6ms/step - loss: 3868.5425 - mae: 3868.5425 - val_loss: 3898.0532 - val_mae: 3898.0532
Epoch 7/80
186/186 [==============================] - 1s 6ms/step - loss: 3849.3962 - mae: 3849.3962 - val_loss: 4249.1685 - val_mae: 42

Compiling the model

In [27]:


# Get predictions on the test set
y_test_pred = model.predict(X_test1)  # Flatten to ensure 1D array

# Compute R² score on test set
r2_test = r2_score(y_test, y_test_pred)
print("R² Score on Test Data:", r2_test)


844/844 [==============================] - 2s 2ms/step
R² Score on Test Data: -0.08719544639647703


In [28]:


# Get predictions on validation data (already a NumPy array)
y_val_pred = model.predict(X_val_split)  # Ensure it's a 1D array

# Compute R² score
r2 = r2_score(y_val_split, y_val_pred)
print("R² Score on Validation Data:", r2)


79/79 [==============================] - 0s 2ms/step
R² Score on Validation Data: -0.10952527429200631


In [25]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [26]:
rf = RandomForestRegressor()
model=rf.fit(X_train_split,y_train_split)

In [27]:
y_pred2 = rf.predict(X_test1)

In [28]:
r2=rf.score(X_test1,y_test)
print('The R-square for Random Forest is: ', r2)

The R-square for Random Forest is:  0.9583720674711406
